## Load the goodreads data as a panda series dataframe


In [5]:
import pandas as pd
from rake_nltk import Rake
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings

from textblob import TextBlob
from textblob import Word

import seaborn as sns

import re
warnings.filterwarnings("ignore")
books = pd.DataFrame(columns = pd.read_csv('/Users/mariamihaila/Downloads/goodreads/book1000k-1100k.csv', index_col = 'Id').columns)
books

,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews


In [2]:
import os
for dirname, _, filenames in os.walk('/Users/mariamihaila/Downloads/goodreads/'):
    for filename in filenames:
        if "_" not in filename:
            books = pd.concat([books, pd.read_csv(os.path.join(dirname, filename), index_col = 'Id')])
            print(os.path.join(dirname, filename), 'OK')


/Users/mariamihaila/Downloads/goodreads/book1900k-2000k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1600k-1700k.csv OK
/Users/mariamihaila/Downloads/goodreads/book700k-800k.csv OK
/Users/mariamihaila/Downloads/goodreads/book600k-700k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1000k-1100k.csv OK
/Users/mariamihaila/Downloads/goodreads/book100k-200k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1300k-1400k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1500k-1600k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1200k-1300k.csv OK
/Users/mariamihaila/Downloads/goodreads/book800k-900k.csv OK
/Users/mariamihaila/Downloads/goodreads/book2000k-3000k.csv OK
/Users/mariamihaila/Downloads/goodreads/book400k-500k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1700k-1800k.csv OK
/Users/mariamihaila/Downloads/goodreads/book1800k-1900k.csv OK
/Users/mariamihaila/Downloads/goodreads/book200k-300k.csv OK
/Users/mariamihaila/Downloads/goodreads/book500k-600k.csv OK
/Users

### Exploratory Data Analysis


In [3]:
books.head()

,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
1900511,Barbarossa,Christopher Ailsby,1840138009,3.0,2007,4,1,New Line Books,5:0,4:0,3:1,2:0,1:0,total:1,0,NaN,NaN,"On 22 June 1941, Adolf Hitler launched Operati...",NaN,192.0
1900512,Collector's Guide to German World War II: Comb...,Christopher Ailsby,0781802253,0.0,1994,7,1,Hippocrene Books,5:0,4:0,3:0,2:0,1:0,total:0,0,NaN,NaN,NaN,NaN,160.0
1900514,Images of Barbarossa,Christopher Ailsby,0711028257,3.5,2001,1,25,Ian Allan Ltd,5:0,4:5,3:2,2:1,1:0,total:8,0,NaN,NaN,"On 22 June 1941, Adolf Hitler launched Operati...",NaN,256.0
1900520,Romania After 2000: Five New Romanian Plays,Daniel Charles Gerould,0595436560,4.0,2007,9,1,Martin E. Segal Theatre Center Publications,5:1,4:4,3:1,2:0,1:0,total:6,0,NaN,NaN,The first anthology of new Romanian Drama publ...,NaN,226.0
1900521,Global Foreigners: An Anthology of Plays,Saviana Stănescu,1905422423,4.6,2006,12,7,Seagull Books,5:4,4:0,3:1,2:0,1:0,total:5,0,NaN,NaN,"In Waxing West, Daniella, newly arrived in the...",NaN,320.0


In [4]:
books.sort_index(inplace = True)

In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1850310 entries, 1 to 4846451
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Name                   object 
 1   Authors                object 
 2   ISBN                   object 
 3   Rating                 float64
 4   PublishYear            object 
 5   PublishMonth           object 
 6   PublishDay             object 
 7   Publisher              object 
 8   RatingDist5            object 
 9   RatingDist4            object 
 10  RatingDist3            object 
 11  RatingDist2            object 
 12  RatingDist1            object 
 13  RatingDistTotal        object 
 14  CountsOfReview         object 
 15  Language               object 
 16  pagesNumber            object 
 17  Description            object 
 18  Count of text reviews  object 
 19  PagesNumber            float64
dtypes: float64(2), object(18)
memory usage: 296.5+ MB


How many unique book title are there? 


>  Answer: 1636235




In [6]:
books['Name'].nunique()

1636235

How many books are in the dataset? 


> Answer: 1850310

This means that some of the books are probably duplicates of eachother, possibly different editions. Let's remove all the title duplicates. 



In [7]:
len(books['Name'])

1850310

## Removing duplicate books from the dataset

Problems I ran into: 
Two columns of the data frame - Authors and Publisher - contain lists, an 'unhashable type.' To get around this problem, make sure to specify that you are dropping duplicates from the 'Name' column only by setting the subset argument to 'Name'

In [8]:
books.sort_index(inplace = True)


In [9]:
# extract duplicate rows
books.loc[books.Name.duplicated(), :]
# drop the duplicate books from the data set
books.drop_duplicates(subset = ['Name'], inplace = True)

Make sure the duplicate drop was succesful: 

In [10]:
(books.Name.duplicated()).sum()

0

In [11]:
len(books['Name'])

1636235

#### Null values
Check for null values. We see that there are quite a few entries with data missing. For the sake of accurate reccomendation, I will remove the 679122 books that are missing a description. 

In [12]:
books.isnull().sum()

Name                           0
Authors                        0
ISBN                        5013
Rating                         0
PublishYear                    0
PublishMonth                   0
PublishDay                     0
Publisher                  16634
RatingDist5                    0
RatingDist4                    0
RatingDist3                    0
RatingDist2                    0
RatingDist1                    0
RatingDistTotal                0
CountsOfReview                 0
Language                 1425579
pagesNumber               881824
Description               638645
Count of text reviews    1282821
PagesNumber               754411
dtype: int64

In [13]:
books[books['Description'].isnull()].head()

,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,NaN,4.57,2006,16,9,Scholastic Inc.,5:1546466,4:556485,3:159960,2:25317,1:9896,total:2298124,28062,eng,652,NaN,NaN,NaN
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,0439358078,4.50,2004,1,9,Scholastic Inc.,5:1493113,4:604283,3:211781,2:37005,1:12455,total:2358637,29770,eng,870,NaN,NaN,NaN
3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,NaN,4.47,2003,1,11,Scholastic Inc,5:4268227,4:1513191,3:567458,2:130310,1:108202,total:6587388,75911,eng,309,NaN,NaN,NaN
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,0439554896,4.42,2003,1,11,Scholastic,5:1504505,4:706082,3:288821,2:49353,1:11896,total:2560657,244,eng,352,NaN,NaN,NaN
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,043965548X,4.57,2004,1,5,Scholastic Inc.,5:1749958,4:630534,3:194848,2:24849,1:10128,total:2610317,37093,eng,435,NaN,NaN,NaN


In [14]:
books.dropna(subset = ['Description'], inplace = True)

In [15]:
# check to see if succesful
books.Description.isnull().sum()

0

### Data preprocessing

using regex and nltk
- convert all text to lowercase 
- remove white space from description
- remove punctuation from description
- remove stop words from description
- lemmatize words in description (convert all words to their root)

TIPS: make a copy of the dataframe because you WILL inevitably mess up with regex and have to rerun everything. 


In [16]:
# make a copy of the dataframe for when I inevitably will mess up
df = pd.Series()
df['Description'] = books['Description']

In [17]:
# remove punctuation
df['Description'] = df['Description'].str.replace('[^\w\s]','')

In [18]:
punctuation_removed_test = df['Description'][600001]

In [19]:
# remove stop words. 
stop = stopwords.words('english')
df['Description'] = df['Description'].astype(str).apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [20]:
stopwords_removed_test = df['Description'][600001]
print("o.g: ", books['Description'][600001])
print("stopwords_removed_test: ", stopwords_removed_test)

o.g:  At the age of fifteen, Jennifer Rothschild confronted two unshakable realities: Blindness is inevitable ... and God is enough. Now this popular author, speaker, and recording artist offers poignant lessons that illuminate a path to freedom and fulfillment. With warmth, humor, and insight, Jennifer shares the guiding principles she walks by -- and shows you how to walk forward by faith into God's marvelous light
stopwords_removed_test:  At age fifteen Jennifer Rothschild confronted two unshakable realities Blindness inevitable God enough Now popular author speaker recording artist offers poignant lessons illuminate path freedom fulfillment With warmth humor insight Jennifer shares guiding principles walks shows walk forward faith Gods marvelous light


In [21]:
# convert text to lowercase
df['Description'] = df['Description'].astype(str).str.lower() 

In [22]:
lowercase_conversion_test = df['Description'][600001]
print("o.g: ", books['Description'][600001])
print("stopwords_removed_test: ", stopwords_removed_test)
print("lowercase test: ", lowercase_conversion_test)

o.g:  At the age of fifteen, Jennifer Rothschild confronted two unshakable realities: Blindness is inevitable ... and God is enough. Now this popular author, speaker, and recording artist offers poignant lessons that illuminate a path to freedom and fulfillment. With warmth, humor, and insight, Jennifer shares the guiding principles she walks by -- and shows you how to walk forward by faith into God's marvelous light
stopwords_removed_test:  At age fifteen Jennifer Rothschild confronted two unshakable realities Blindness inevitable God enough Now popular author speaker recording artist offers poignant lessons illuminate path freedom fulfillment With warmth humor insight Jennifer shares guiding principles walks shows walk forward faith Gods marvelous light
lowercase test:  at age fifteen jennifer rothschild confronted two unshakable realities blindness inevitable god enough now popular author speaker recording artist offers poignant lessons illuminate path freedom fulfillment with warmt

In [23]:
# lemmatizing - convert all words to their root. This will take some time.

nltk.download('wordnet')
df['Description'] = df['Description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['Description'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariamihaila/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


600000    lesson learnedbr br coordinating publicity tou...
600001    at age fifteen jennifer rothschild confronted ...
600004    ithe blended learning booki user manual implem...
600005    blessons learned success achievedb help motiva...
600007    many great advance technology resulted risky e...
Name: Description, dtype: object

In [24]:
lemmatized_test = df['Description'][600001]
print("o.g: ", books['Description'][600001])
print("stopwords_removed_test: ", stopwords_removed_test)
print("lowercase test: ", lowercase_conversion_test)
print("lemmatized test: ", lemmatized_test)


o.g:  At the age of fifteen, Jennifer Rothschild confronted two unshakable realities: Blindness is inevitable ... and God is enough. Now this popular author, speaker, and recording artist offers poignant lessons that illuminate a path to freedom and fulfillment. With warmth, humor, and insight, Jennifer shares the guiding principles she walks by -- and shows you how to walk forward by faith into God's marvelous light
stopwords_removed_test:  At age fifteen Jennifer Rothschild confronted two unshakable realities Blindness inevitable God enough Now popular author speaker recording artist offers poignant lessons illuminate path freedom fulfillment With warmth humor insight Jennifer shares guiding principles walks shows walk forward faith Gods marvelous light
lowercase test:  at age fifteen jennifer rothschild confronted two unshakable realities blindness inevitable god enough now popular author speaker recording artist offers poignant lessons illuminate path freedom fulfillment with warmt

In [25]:
# make sure df, the dataframe copy, is still intact
df['Description']

600000     lesson learnedbr br coordinating publicity tou...
600001     at age fifteen jennifer rothschild confronted ...
600004     ithe blended learning booki user manual implem...
600005     blessons learned success achievedb help motiva...
600007     many great advance technology resulted risky e...
                                 ...                        
4846428    in battle human microbe knowledge may best wea...
4846434    excerpt meditation life passion christ from br...
4846447    this text combination 32 reading 10 case it or...
4846448    throughout working life vincent mulchrone reco...
4846449    journalist broadcaster author ian skidmore col...
Name: Description, Length: 997590, dtype: object

In [26]:
# now replace books['Description'] with the cleaned df['Description']
# actually to play it safe ... make a new column.  
books['Description_cleaned'] = df['Description']


### Create the book corpus 
combine the author, published and cleaned description columns

In [27]:
books['text'] = books['Authors'].map(str) + ' ' + books['Publisher'] + ' ' + books['Description_cleaned']

In [28]:
books['text'][600001]

'Jennifer Rothschild Lifeway Church Resources at age fifteen jennifer rothschild confronted two unshakable reality blindness inevitable god enough now popular author speaker recording artist offer poignant lesson illuminate path freedom fulfillment with warmth humor insight jennifer share guiding principle walk show walk forward faith god marvelous light'

In [29]:
books_corpus = books[['Name', 'ISBN','text']]
books_corpus = books_corpus.fillna(" ")
books_corpus['text'] = books_corpus['text'].astype(str).str.lower() # convert everything to lowercase
books_corpus['text'] = books_corpus['text'].str.replace('[^\w\s]','') # remove punctuation

In [30]:
books_corpus

,Name,ISBN,text
600000,"Lessons Learned (Great Chefs, #2)",037351025X,nora roberts silhouette lesson learnedbr br co...
600001,Walking by Faith: Lessons Learned in the Dark,0633099325,jennifer rothschild lifeway church resources a...
600004,"The Blended Learning Book: Best Practices, Pro...",0787972967,josh bersin pfeiffer ithe blended learning boo...
600005,Lessons Learned: Successes Achieved: Be Prepar...,0595417566,robert g gillio iuniverse blessons learned suc...
600007,Computing Calamities: Lessons Learned from Pro...,0130828629,robert l glass prentice hall many great advanc...
...,...,...,...
4846428,"Disinfection and Decontamination: Principles, ...",0849390745,gurusamy manivannan crc press in battle human ...
4846434,Meditations on the Life and Passion of Christ:...,0527001554,charlotte develyn krause publications excerpt ...
4846447,International and Global Marketing: Concepts a...,0256218943,taylor w meloan irwinmcgrawhill this text comb...
4846448,The Best of Vincent Mulchrone,0955823811,vincent mulchrone revel barker throughout work...


In [31]:
# DONT FORGET TO SAVE THE CLEANED DATA
compression_opts = dict(method='zip', archive_name='books_corpus.csv')  # doctest: +SKIP
books_corpus.to_csv('goodreads_recc.zip', index=False, compression=compression_opts)  # doctest: +SKIP

## Time for TF-IDF

In [2]:
import pandas as pd
books_corpus = pd.read_csv('books_corpus.csv')

In [3]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_books = tfidf_vectorizer.fit_transform((books_corpus['text'][:10000])) 
tfidf_books

<10000x75631 sparse matrix of type '<class 'numpy.float64'>'
	with 659327 stored elements in Compressed Sparse Row format>

### Building the recommendation system:

- compute cosine similarity using tfidf
- the recommendation function:

    *    get index of the user inputted book title
    *    search cosine similarity matrix, sorting the similarity values in descending order
    * return a list of the top 10 most similar books
    




In [4]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_books)

In [5]:
cosine_sim.shape

(10000, 10000)

In [6]:
 indices = pd.Series(books_corpus['Name'])

 def recommend(title, cosine_sim = cosine_sim):
    recommended_books = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar books
    
    for i in top_10_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_books.append(list(books_corpus['Name'])[i])
        
    return recommended_books

In [8]:
books_corpus['Name'][:20000]

0                        Lessons Learned (Great Chefs, #2)
1            Walking by Faith: Lessons Learned in the Dark
2        The Blended Learning Book: Best Practices, Pro...
3        Lessons Learned: Successes Achieved: Be Prepar...
4        Computing Calamities: Lessons Learned from Pro...
                               ...                        
19995                                  The Seven Year Itch
19996               Philadelphia Murals  Stories They Tell
19997                                 Philadelphia Stories
19998                                           Odd Couple
19999                  Noel Coward Collected Short Stories
Name: Name, Length: 20000, dtype: object

In [12]:
recommend("Economists' Mathematical Manual")

['The Age of the Economist',
 'Measure, Integral and Probability',
 'Playing with Infinity',
 'Anesthetic and Obstetric Management of High-Risk Pregnancy',
 'Political Anthropology: An Introduction',
 'Economics: Making sense of the Modern Economy',
 'The Instant Economist: All The Basic Principles Of Economics In 100 Pages Of Plain Talk',
 'Numbers Guide: The Essentials of Business Numeracy',
 "Chef's Book of Formulas, Yields, and Sizes",
 'On the Other Hand: Essays on Economics, Economists, and Politics']